In [1]:
from bs4 import BeautifulSoup
import requests
import re
import mysql.connector
import datetime

In [ ]:
conn = mysql.connector.connect(
    host = "*****",
    port = "****",
    user = "****",
    password = "****",
    database = "****"
)

cursor = conn.cursor(buffered=True)

In [63]:
conn.close()

수집데이터: 
*(공고url)*, 주요업무, 우대사항, 근무환경, 복리후생, *(공고게시일)*
*(회사id, 회사명, 회사형태, 산업)*, 사원수, *(설립일)*, 매출액, *(회사홈페이지)*, 평균연봉

In [6]:
begin_url = "https://www.saramin.co.kr/zf_user/search?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword=%EB%A1%9C%EB%B4%87+%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4+%EA%B0%9C%EB%B0%9C&recruitPage={}&recruitSort=reg_dt&recruitPageCount=40&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=y"
post_base_url = "https://www.saramin.co.kr/zf_user/jobs/view?rec_idx="

res = requests.get(begin_url.format(1), headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.text, 'html.parser')

cnt = re.search(r'\d+', soup.find("section", id="recruit_info").find("span", class_="cnt_result").string).group()
index_num = int(int(cnt) / 40) + 1



for i in range(1,index_num+1):
    res = requests.get(begin_url.format(i), headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, "html.parser")

    # i 페이지 공고id들
    recruit_info_list = soup.find("div", id="recruit_info_list").find_all("div", class_=re.compile(r"item_recruit"))

    for recruit in recruit_info_list:
        post_url = post_base_url + recruit.get("value")
        print(recruit.get("value"))
        res = requests.get(post_url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.text, "html.parser")
        # 공고 게시일 있는 div
        info_period = soup.find("dl", class_="info_period")
        # 회사id, 회사명, 회사형태, 산업, 사원수, 설립일, 회사홈페이지 있는 div
        info = soup.find("div", class_="wrap_info")

        posted_date = comp_sn = comp_name = comp_form = sector = establish = comp_homepage = None

        pd = info_period.find("dt", string="시작일").find_next_sibling("dd").get_text()
        posted_date = datetime.datetime.strptime(pd, "%Y.%m.%d %H:%M")
        print(posted_date)

        if info:
            tit_area_a = info.find("div", class_="tit_area").find("a")
            comp_name = tit_area_a.string.strip().replace(" ", "")
            print(comp_name)
            csn = tit_area_a.get("href")
            comp_sn = re.search(r"csn=([^&]+)", csn).group(1)
            print(comp_sn)
            
            for dl in info.find("div", class_="info_area").find_all("dl"):
                dt_text = dl.find("dt").get_text(strip=True)
                if "기업형태" in dt_text:
                    comp_form = dl.find("dd").get("title")
                elif "업종" in dt_text:
                    sector = dl.find("dd").get_text(strip=True)
                    print(sector)
                elif "설립일" in dt_text:
                    ed = dl.find('dd').contents[0].strip()
                    print(ed)
                    try:
                        establish = datetime.datetime.strptime(ed, "%Y년 %m월 %d일")
                        print(establish)
                    except ValueError:
                        print("ValueError")
                        establish = None
                elif "홈페이지" in dt_text:
                    comp_homepage = dl.find("dd").get("title")
                    print(comp_homepage)
        else:
            comp_sn = comp_name = comp_form = sector = establish = comp_homepage = None
        
        # cursor.execute("""insert into saramin 
        #         (id, url, 기업명, 기업형태, 산업업종, 사원수, 설립연도, 매출액)
        #         values
        #         (%s,%s,%s,%s,%s,%s,%s,%s)""",
        #         (recruit.get("value"), post_url, comp_name, comp_form, industry, employee_num, established_date, profit))



49090385
2024-09-27 10:00:00
(주)아데코코리아
S0hPQnFNWTdWS1Q4WjMxaFBDOFJJdz09
상용 인력 공급 및 인사관리 서비스업
1999년 5월 6일
1999-05-06 00:00:00
www.adecco.co.kr
49089775
2024-09-27 09:00:00
(주)벰로보틱스
K1RVeXZicmt6YlJuTzNyRkl0UGtadz09
시스템 소프트웨어 개발 및 공급업
2017년 1월 13일
2017-01-13 00:00:00
www.bemsoft.com
49089618
2024-09-27 09:00:00
(주)시즐
Y3VRdjFtRHlXZjJuQlRhOWM5dlNCZz09
컴퓨터 프로그래밍 서비스업
2016년 8월 4일
2016-08-04 00:00:00
https://sizl.co.kr/
49089599
2024-09-27 09:00:00
(주)시즐
Y3VRdjFtRHlXZjJuQlRhOWM5dlNCZz09
컴퓨터 프로그래밍 서비스업
2016년 8월 4일
2016-08-04 00:00:00
https://sizl.co.kr/
49089574
2024-09-27 09:00:00
(주)피엠에프
amFwR1JFMFBoVG9ib0ZaSTFZbDRUZz09
전기회로 접속장치 제조업
2022년 9월 19일
2022-09-19 00:00:00
https://www.pmotionf.com/
49089453
2024-09-27 09:00:00
(주)휴먼트리월드와이드
NGJqRDg5S0pDbkpCNjdXbE15bUIxdz09
경영 컨설팅업
2011년 6월 24일
2011-06-24 00:00:00
49089184
2024-09-27 09:00:00
다임리서치(주)
clI2cFZ4WCtBaWZiSnJKN2l2RlZ2Zz09
컴퓨터시스템 통합 자문 및 구축 서비스업
2020년 2월 5일
2020-02-05 00:00:00
www.daimresearch.com/
49088833
2024-09-27 08:00:00
탑커리어인사이트
akxW

KeyboardInterrupt: 

In [67]:
conn.commit()

In [76]:
cursor.execute("select id from saramin where 기업명 is null")
result = cursor.fetchall()

for r in result:
    print(r[0])
    post_url = post_base_url + r[0]
    res = requests.get(post_url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(res.text, "html.parser")
    company_name = soup.find("div", class_="jv_header").find("a", class_="company")
    print(company_name["title"])
    
    cursor.execute("update saramin set 기업명=%s where id=%s", (company_name["title"], r[0]))


48706971
(주)알지티
48742473
(주)제트웨이크
48758127
(주)파이엇
48772578
(주)엑스와이지
48814396
비엠티
48891187
(주)알지티
48910225
(주)크래블
48912263
(주)엑스와이지
48948767
(주)엑스와이지
48948805
(주)엑스와이지
48962556
(주)모빌리티원
48970872
(주)알지티
48978846
(주)휴톰
48986185
(주)에스티에스로보테크
49010281
(주)오픈에스지
49011086
(주)크래블
49043546
(주)엑스와이지
49046610
에이치알포유(주)
49048528
에이치알포유(주)
49052331
(주)비전나비로보틱스코리아
49052634
(주)토르드라이브
49052641
(주)토르드라이브
49052654
(주)토르드라이브
49074572
에이치알포유(주)
49074596
에이치알포유(주)
49076374
에이치알포유(주)


In [77]:
conn.commit()